In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mse-pysupport-hackathon/MetaData.xlsx
/kaggle/input/mse-pysupport-hackathon/holdout.csv
/kaggle/input/mse-pysupport-hackathon/simulation.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('/kaggle/input/mse-pysupport-hackathon/simulation.csv')
X_test_full = pd.read_csv('/kaggle/input/mse-pysupport-hackathon/holdout.csv', index_col='Id')

In [3]:
X.head()

,SalePrice,SaleDate,SaleDeed,ArmsLengthTransaction,EstimateLand,EstimateBuilding,MostRecentSale,Pin,PropertyAddress,NeighborhoodCode,...,RepairCondition,GarageIndicator,Garage1Size,Garage1Material,Garage1Attachment,Garage1Area,Garage2Size,Garage2Material,Garage2Attachment,Garage2Area
0,407000,09/17/2013,1326022056,1,55800,80200,1.0,13013260100000,5627 N RICHMOND ST,42,...,Avg,1.0,3.0,Wood,No,No,7.0,Other,Other,Other1
1,1,05/08/2017,1712801037,0,30620,34480,1.0,14061120241008,6116 N CLAREMONT AVE,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,340000,07/17/2013,1319841023,1,8490,253800,0.0,14213140481165,3200 N LAKE SHORE DR,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89000,06/29/2018,1818057130,1,23760,124690,1.0,24362310030000,2449 NEW ST,132,...,Avg,1.0,3.0,Wood,No,No,7.0,Other,Other,Other1
4,375000,09/24/2014,1426755105,1,59850,253540,1.0,16081150120000,229 N TAYLOR AVE,90,...,Avg,1.0,3.0,Wood,No,No,7.0,Other,Other,Other1


In [4]:
# Feature engineering

X["PastMarketValue"] = X["EstimateLand"] + X["EstimateBuilding"]

X["Total_sq_ft"] = X["LandSquareFeet"] + X["BuildingSquareFeet"]

X["SquareFeetRatio"] = X["BuildingSquareFeet"] / X["Total_sq_ft"]



In [5]:
X_test_full["PastMarketValue"] = X_test_full["EstimateLand"] + X_test_full["EstimateBuilding"]

X_test_full["Total_sq_ft"] = X_test_full["LandSquareFeet"] + X_test_full["BuildingSquareFeet"]

X_test_full["SquareFeetRatio"] = X_test_full["BuildingSquareFeet"] / X_test_full["Total_sq_ft"]

In [6]:
# Dropping columns
drop_cols = ["EstimateLand","EstimateBuilding"]

X.drop(columns = drop_cols,axis = 1,inplace = True)



In [7]:
drop_cols_2 = ["Total_sq_ft","LandSquareFeet","BuildingSquareFeet"]

X.drop(columns = drop_cols_2,axis = 1,inplace = True)


In [8]:
X_test_full.drop(columns = drop_cols,axis = 1,inplace = True)

In [9]:
X_test_full.drop(columns = drop_cols_2,axis = 1,inplace = True)

In [10]:
X.shape,X_test_full.shape

((408359, 62), (143177, 61))

In [11]:
# Separate Year and Month from SaleDate

X['SaleYear'] = pd.DatetimeIndex(X['SaleDate']).year

X_test_full['SaleYear'] = pd.DatetimeIndex(X_test_full['SaleDate']).year

In [12]:
# SaleMonth can also indicate demand
X['SaleMonth'] = pd.DatetimeIndex(X['SaleDate']).month

X_test_full['SaleMonth'] = pd.DatetimeIndex(X_test_full['SaleDate']).month

In [13]:
X.head()

,SalePrice,SaleDate,SaleDeed,ArmsLengthTransaction,MostRecentSale,Pin,PropertyAddress,NeighborhoodCode,TownshipLocation,TownshipName,...,Garage1Attachment,Garage1Area,Garage2Size,Garage2Material,Garage2Attachment,Garage2Area,PastMarketValue,SquareFeetRatio,SaleYear,SaleMonth
0,407000,09/17/2013,1326022056,1,1.0,13013260100000,5627 N RICHMOND ST,42,City,Jefferson,...,No,No,7.0,Other,Other,Other1,136000,0.147102,2013,9
1,1,05/08/2017,1712801037,0,1.0,14061120241008,6116 N CLAREMONT AVE,11,City,Lake View,...,NaN,NaN,NaN,NaN,NaN,NaN,65100,NaN,2017,5
2,340000,07/17/2013,1319841023,1,0.0,14213140481165,3200 N LAKE SHORE DR,62,City,Lake View,...,NaN,NaN,NaN,NaN,NaN,NaN,262290,NaN,2013,7
3,89000,06/29/2018,1818057130,1,1.0,24362310030000,2449 NEW ST,132,South,Worth,...,No,No,7.0,Other,Other,Other1,148450,0.270718,2018,6
4,375000,09/24/2014,1426755105,1,1.0,16081150120000,229 N TAYLOR AVE,90,South,Oak Park,...,No,No,7.0,Other,Other,Other1,313390,0.150943,2014,9


In [14]:
X.describe()

,SalePrice,SaleDeed,ArmsLengthTransaction,MostRecentSale,Pin,NeighborhoodCode,Longitude,Latitude,OHareNoise,Floodplain,...,CentralAir,Fireplaces,OtherImprovements,GarageIndicator,Garage1Size,Garage2Size,PastMarketValue,SquareFeetRatio,SaleYear,SaleMonth
count,4.083590e+05,4.083590e+05,408359.000000,408095.000000,4.083590e+05,408359.000000,403078.000000,403078.000000,403078.000000,403078.000000,...,286955.000000,286955.000000,286955.000000,286955.000000,286955.000000,286955.000000,4.083590e+05,286955.000000,408359.000000,408359.000000
mean,2.278922e+05,1.650681e+09,0.818052,0.757184,1.594402e+13,100.452803,-87.763843,41.874265,0.013772,0.039533,...,0.446840,0.299273,14.923148,0.824530,2.239463,6.997498,2.533165e+05,0.256594,2016.313545,6.624064
std,4.023689e+05,2.027310e+08,0.385802,0.428785,7.447435e+12,97.250955,0.143830,0.159546,0.116541,0.194860,...,0.497167,0.535897,516.151438,0.380369,1.376879,0.112010,3.203531e+05,0.134322,1.979928,3.274229
min,0.000000e+00,6.234020e+08,0.000000,0.000000,1.011000e+12,10.000000,-88.263497,41.469868,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,2013.000000,1.000000
25%,5.100000e+04,1.508442e+09,1.000000,1.000000,1.201104e+13,31.000000,-87.827509,41.759186,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,7.000000,1.232000e+05,0.159247,2015.000000,4.000000
50%,1.650000e+05,1.635810e+09,1.000000,1.000000,1.527422e+13,71.000000,-87.728092,41.901550,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,3.000000,7.000000,1.910700e+05,0.219432,2016.000000,7.000000
75%,2.999000e+05,1.819319e+09,1.000000,1.000000,2.017221e+13,130.000000,-87.660191,41.995833,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,3.000000,7.000000,2.974900e+05,0.331210,2018.000000,9.000000
max,1.350005e+08,7.901008e+09,1.000000,1.000000,3.332302e+13,600.000000,-87.524884,42.153979,1.000000,1.000000,...,1.000000,2.000000,74200.000000,1.000000,7.000000,7.000000,1.209965e+07,0.936838,2019.000000,12.000000


In [15]:
# Separate Target from features

y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

In [16]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, 
                                                                train_size=0.7, test_size=0.3,
                                                                random_state=0)

In [17]:
# Select categorical columns with relatively cardinality

categorical_cols = [col for col in X_train_full.columns 
                    if X_train_full[col].nunique() < 10 and 
                    X_train_full[col].dtype == "object"]

In [18]:
categorical_cols

['TownshipLocation',
 'Basement',
 'BasementFinish',
 'CentralHeating',
 'OtherHeating',
 'AtticType',
 'AtticFinish',
 'WallMaterial',
 'RoofMaterial',
 'CathedralCeiling',
 'DesignPlan',
 'ConstructionQuality',
 'Renovation',
 'Porch',
 'RepairCondition',
 'Garage1Material',
 'Garage1Attachment',
 'Garage1Area',
 'Garage2Material',
 'Garage2Attachment',
 'Garage2Area']

In [19]:
# select numerical features

numerical_cols = [col for col in X_train_full.columns if 
                X_train_full[col].dtype in ['int64', 'float64']]

In [20]:
len(numerical_cols)

37

In [21]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [22]:
X_train.shape,X_valid.shape

((285851, 58), (122508, 58))

In [23]:
## Preprocess the data and train the model in a pipeline


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [24]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = XGBRegressor()

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)


print('R-squared:', r2_score(y_valid, preds))

R-squared: 0.3586749249286334


In [25]:
## Trying to improve the model

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy = "median") # Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps = [("imputer",SimpleImputer(strategy = "most_frequent")),
                                            ("one_hot",OneHotEncoder(handle_unknown = "ignore"))]) # Your code here

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# Define model

model = XGBRegressor()

In [26]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['SaleDeed',
                                                   'ArmsLengthTransaction',
                                                   'MostRecentSale', 'Pin',
                                                   'NeighborhoodCode',
                                                   'Longitude', 'Latitude',
                                                   'OHareNoise', 'Floodplain',
                                                   'RoadProximity',
                                                   'CensusTract', 'Apartments',
                                                   'Age', 'TypeOfResidence',
                                                   'MultiPropertyIndicator',
                                                   'MultiFamilyIndicator',
        

In [27]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test)

In [28]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission_Sreyartha_0.csv', index=False)